<a href="https://colab.research.google.com/github/janithcyapa/energy-plus-utility/blob/main/energyplus-utility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EnergyPlus Utility

## Installation From Github Repo and Setup Environment

In [ ]:
!pip uninstall -y energy-plus-utility

In [1]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


`prepare_colab_eplus` - Download and Install EnergyPlus 25-1 in `/root/EnergyPlus-25-1-0`

In [2]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")



✅ Installed 'energy-plus-utility' version: 0.2.1+16


In [3]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

# Usage

In [4]:
from eplus.core import EPlusUtil
import types

### Setup Basics

In [37]:
# In your main script or core.py
# verbose, 0 - no logging, 1- verbose logging, 2 - debug logging
OUT_DIR = "/simulation/eplus_out"
sim = EPlusUtil(verbose=2, out_dir=OUT_DIR)

# Reset state before setting model
sim.reset_state()
# Delete previous output directory
sim.delete_out_dir()
# Clear previous outputs
sim.clear_eplus_outputs(patterns="eplusout.*")

Initialized StateMixin
Initialized EnergyPlus State.
Initialized IDFMixin
Initialized LoggingMixin
Initialized SimulationMixin
Initialized UtilsMixin
Initialized HandlersMixin
Initialized SQLMixin
Initialized ControlMixin
Initialized OccupancyMixin
EnergyPlus state has been reset.
Deleted output directory: /simulation/eplus_out


### Set Model using local files

In [38]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
# Set the Model for Simulation
sim.set_model(IDF, EPW)

EnergyPlus state has been reset.
Model set: IDF='/root/EnergyPlus-25-1-0/ExampleFiles/5ZoneAirCooled.idf', EPW='/root/EnergyPlus-25-1-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw', OUT_DIR='/simulation/eplus_out'


### Set Model From URL

In [6]:
url_idf = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/MultizoneOffice.idf"
url_epw = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/LKA_Colombo-Katunayake.434500_SWERA.epw"

# Set the Model for Simulation
sim.set_model_from_url(url_idf, url_epw)

EnergyPlus state has been reset.
Model set: IDF='/simulation/eplus_out/MultizoneOffice.idf', EPW='/simulation/eplus_out/LKA_Colombo-Katunayake.434500_SWERA.epw', OUT_DIR='/simulation/eplus_out'


### View IDF/API Data

In [8]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False) # Need to run this to get catalog

EnergyPlus state has been reset.


0

In [ ]:
sim.api_catalog_df()

In [ ]:
sim.list_available_variables()

In [ ]:
sim.list_available_meters()

In [ ]:
sim.list_available_actuators()

In [ ]:
objects_types = sim.get_idf_object_types()

print(f"{'OBJECT TYPE':<40} | {'COUNT'}")
print("-" * 50)
for obj_type, count in objects_types:
    print(f"{obj_type:<40} | {count}")

In [ ]:
object = sim.extract_idf_objects("Zone")
print(object)

In [42]:
sim.clear_patched_idf()

### SetupSQL

In [50]:
sim.ensure_output_sqlite()

EnergyPlus state has been reset.


'/simulation/eplus_out/5ZoneAirCooled__annual_co2_minimal_clean__sqlite__sqlite__sqlite.idf'

In [51]:
sim.ensure_output_variables([
    {"name": "System Node Temperature", "key": "*", "freq": "Hourly"},
    {"name": "System Node Mass Flow Rate", "key": "*", "freq": "Hourly"},
    {"name": "System Node Humidity Ratio", "key": "*", "freq": "Hourly"},
    {"name": "Zone Air Temperature", "key": "*", "freq": "Hourly"},
    {"name": "Zone Air Relative Humidity", "key": "*", "freq": "Hourly"}
])

sim.ensure_output_variables([
    {"name": "Site Outdoor Air Drybulb Temperature", "key": "Environment"},
    {"name": "Site Outdoor Air Humidity Ratio", "key": "Environment"},
    {"name": "Site Wind Speed", "key": "Environment"},
    {"name": "Site Direct Solar Radiation Rate per Area", "key": "Environment"}
])

EnergyPlus state has been reset.
EnergyPlus state has been reset.


'/simulation/eplus_out/5ZoneAirCooled__annual_co2_minimal_clean__sqlite__sqlite__sqlite__vars__vars.idf'

In [52]:
sim.ensure_output_meters([
    "Electricity:Facility",          # Total building electricity
    "Fans:Electricity",              # Electricity used by HVAC fans
    "Cooling:Electricity",           # Electricity used for cooling (Chillers/DX coils)
    "Heating:Electricity",           # Electricity used for heating
    "ElectricityPurchased:Facility"  # Total grid electricity bought
], freq="Hourly")

EnergyPlus state has been reset.


'/simulation/eplus_out/5ZoneAirCooled__annual_co2_minimal_clean__sqlite__sqlite__sqlite__vars__vars__meters.idf'

### Occupency and CO2

You must first use prepare_run_with_co2 to modify your EnergyPlus Input Data File (IDF). Without this, EnergyPlus won't "know" how to track CO₂ or where to look for your outdoor concentration values.Before the simulation loops, you need to register the handlers. The occupancy_handler needs to be "hooked" into the EnergyPlus callback system so it runs at every timestep.

In [43]:
sim.prepare_run_with_co2(
    outdoor_co2_ppm=415.0,           # Starting outdoor concentration
    per_person_m3ps_per_W=3.82e-8,   # CO2 generation rate (default)
)

EnergyPlus state has been reset.


'/simulation/eplus_out/5ZoneAirCooled__annual_co2_minimal_clean.idf'

In [44]:
# Register the random occupancy logic to run at the start of every iteration
sim.register_handlers(
    "after_hvac",
    [{"method_name": "occupancy_handler","key_wargs": {"lam": 33.0, "min": 20, "max": 45, "seed": 123}}],
    clear=False, run_during_warmup=False
)

['occupancy_handler']

In [ ]:
# Use the helper to push this value into the simulation
sim.co2_set_outdoor_ppm(
        value_ppm=415,
        verify=True
    )

### Run Simualtion

In [28]:
sim.enable_runtime_logging()
# sim.disable_runtime_logging()

In [ ]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False)

In [ ]:
sim.run_design_day()

In [53]:
sim.set_simulation_params(

    start=(1, 1),
    end=(12, 31),
    start_day_of_week="Sunday"
)

EnergyPlus state has been reset.


'/simulation/eplus_out/5ZoneAirCooled__annual_co2_minimal_clean__sqlite__sqlite__sqlite__vars__vars__meters__patched.idf'

In [55]:
sim.run_annual()

Deleted output file: /simulation/eplus_out/eplusout.err
EnergyPlus state has been reset.


1

### Usage of Handlers

In [ ]:
# --- STEP A: DEFINE THE CUSTOM FUNCTION ---
# Notice we define it with 'self' as the first argument, just like a class method.
def dr_supervisor_logic(self, state):
    """
    A simple logic that checks the time and prints a message.
    """
    # Just printing for this demo, but you would put control logic here
    time = self.exchange.current_time(state)
    print(f"[SUPERVISOR] Checking status at time: {time:.2f}")

# --- STEP B: INJECT IT  ---
# This binds the function ONLY to this 'sim' object.
# It creates a true method where 'self' is passed automatically.
sim.my_supervisor = types.MethodType(dr_supervisor_logic, sim)


In [ ]:
print("--- 1. REGISTERING ---")
# We register the NAME of the attribute we just attached ("my_supervisor")
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)
print(f"Registered methods: {registered}")

print("\n--- 2. LISTING ---")
# Check what is currently scheduled for the 'begin' hook
current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

print("\n--- 3. DISABLING (Pausing) ---")
# Scenario: It's Winter, we don't need Demand Response.
# We disable the hook so the logic stops running, but we don't delete it.
sim.disable_hook("begin")
print("Hook 'begin' is now DISABLED. (Simulation runs, but supervisor sleeps)")

print("\n--- 4. ENABLING (Resuming) ---")
# Scenario: Summer is back. Turn the logic back on.
sim.enable_hook("begin")
print("Hook 'begin' is now ENABLED. (Supervisor is active again)")

print("\n--- 5. UNREGISTERING (Deleting) ---")
# Scenario: We want to remove this logic entirely to replace it or clean up.
remaining = sim.unregister_handlers(
    "begin",
    ["my_supervisor"] # Name to remove
)
print(f"Unregistered 'my_supervisor'. Remaining handlers: {remaining}")

# --- RUN ---
# sim.run_annual()

In [ ]:
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)

current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

# --- RUN ---
sim.run_annual()

### HVAC Shedule

In [13]:
# Diable HVAC Shedules
sim.override_hvac_schedules(autodetect=True)
# sim.override_hvac_schedules(
#     schedule_names=["Central Fan Avail Sched", "Heating Setpoint Sched"]
# )

In [16]:
sim.release_hvac_schedules()

### Control

In [ ]:
def zone_temp_controller(self, state):
    """
    Simple Bang-Bang or Proportional controller for Zone Temperature.
    """
    # 1. READ: Get the current zone temperature
    # Using your 'runtime_get_variable' helper
    current_temp = self.runtime_get_variable(
        state,
        name="Zone Mean Air Temperature",
        key="CORE_ZN"
    )

    # Safety check: API might not be ready during first few ticks
    if current_temp is None:
        return

    # 2. DECIDE: Logic to determine the new setpoint
    # Example: If it's too hot (> 25C), drop setpoint to 22C. Otherwise, stay at 24C.
    if current_temp > 10.0:
        new_setpoint = 12.0
    else:
        new_setpoint = 14.0

    # 3. WRITE: Update the cooling setpoint schedule
    # Using your 'runtime_set_actuator' helper
    success = self.runtime_set_actuator(
        state,
        component_type="Schedule:Compact",
        control_type="Schedule Value",
        actuator_key="Cooling_Setpoint_Sched",
        value=new_setpoint,
        log=True # This will use your internal _log to record the change
    )

# 4. BIND & REGISTER
# Attach the function to your simulation object
sim.zone_control = types.MethodType(zone_temp_controller, sim)

# Register it to the 'begin_zone_timestep_before_set_point_manager' hook
# This is the best hook for temperature control as it happens right before
# HVAC systems calculate their response for the step.
sim.register_handlers(
    "begin",
    ["zone_control"]
)

sim.run_annual()

### SQL

In [46]:
df_list = sim.list_sql_zone_variables(like="Zone %Temperature%")
print(df_list)

FileNotFoundError: /simulation/eplus_out/eplusout.sql not found. Run ensure_output_sqlite() and a simulation first.

In [ ]:
df_list = sim.list_sql_zone_variables(like="System Node %Temperature%")
print(df_list)

In [47]:
fig = sim.plot_sql_zone_variable(
    var_name="Zone Air Temperature",
    keys=["PERIMETER_ZN_1 ZN", "CORE_ZN ZN"],
    resample="1D",
    start="2002-01-01"
)

FileNotFoundError: /simulation/eplus_out/eplusout.sql not found. Run ensure_output_sqlite() and re-run the sim.

In [12]:
fig = sim.plot_sql_zone_variable(
    var_name="Zone Air Temperature",
    keys=["PERIMETER_ZN_1 ZN", "CORE_ZN ZN"],
    resample="1D",
    start="2002-01-01"
)


In [ ]:
sim.plot_sql_meters(
    ["Electricity:Facility", "Fans:Electricity"],
    resample="1D"  # Look at daily totals
)

In [ ]:
# Define your "Control" (Setpoints) and "Output" (Actual Temps)
controls = [{"kind": "var", "name": "Zone Thermostat Heating Setpoint Temperature", "key": "*"}]
outputs  = [{"kind": "var", "name": "Zone Air Temperature", "key": "*"}]

# Generate a correlation heatmap
sim.plot_sql_cov_heatmap(
    control_sels=controls,
    output_sels=outputs,
    stat="corr",  # Show correlation (-1 to 1)
    resample="1h"
)

In [ ]:
# Export outdoor conditions to a CSV for external analysis
sim.export_weather_sql_to_csv(csv_filename="site_weather_data.csv")

In [ ]:
# Get a raw DataFrame of zone temperatures
df = sim.get_sql_series_dataframe([
    {"kind": "var", "name": "Zone Air Temperature", "key": "*"}
])
display(df)
# Now you can use df.mean(), df.groupby(), etc.

### Test

In [ ]:
args = [
        '-w', EPW,
        '-d', OUT_DIR,
        IDF
    ]

print("Starting simulation...")
exit_code = sim.runtime.run_energyplus(sim.state, args)

if exit_code == 0:
    print("Simulation success!")
else:
    print("Simulation failed!")

In [ ]:
print(dir(sim))
print(hasattr(sim, 'run_design_day'))